In [243]:
import robotic as ry
import matplotlib.pyplot as plt
import numpy as np
import time
pclColor = [0,0,255]
max_trials = 10
max_vel = 100
max_acc = max_vel

C = ry.Config()
C.addFile('/home/vboxuser/Desktop/CS449/Migi/hw3/GFiles-HW3/environment.g')

# C.addFile('./environment.g')
# C.addFile(ry.raiPath('environment.g'))
# qHome = C.getJointState()
# print(qHome)
C.view()
print(ry.raiPath('environment.g'))

/home/vboxuser/Desktop/CS449/rai_venv/lib/python3.8/site-packages/robotic/rai-robotModels/environment.g


# 1.1 Perception Module

In [244]:
numOfCameras = 4

### Get point clouds for each camera

In [245]:
cam = [None] * numOfCameras
rgb = [None] * numOfCameras
depth = [None] * numOfCameras
pcl = [None] * numOfCameras
pclFrames = [None] * numOfCameras


for i in range(numOfCameras):
  cam[i] = ry.CameraView(C)
  cam[i].setCamera('camera'+str(i+1))
  rgb[i], depth[i] = cam[i].computeImageAndDepth(C)
  pcl[i] = ry.depthImage2PointCloud(depth[i], cam[i].getFxycxy())
  print(pcl[i].shape)

(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)


### Now normalise the points relative to world frame

In [246]:
for i in range(numOfCameras):
  curCam = C.getFrame("camera"+str(i+1))  
  curCamRotMat = curCam.getRotationMatrix()
  curCamPos = curCam.getPosition()
    
  for h in range(pcl[i].shape[0]):    # Iterate over height
    for w in range(pcl[i].shape[1]):  # Iterate over width
      point = pcl[i][h, w]            # Get the 3D point [x, y, z]
     
      # Apply rotation
      rotated_point = np.dot(curCamRotMat, point)  # Rotate the point
      
      # Apply translation
      transformed_point = rotated_point + curCamPos  # Add the camera position
      
      pcl[i][h, w] = transformed_point      
      

# # Visualize the point cloud
# pclFrames = [None] * numOfCameras

# for i in range(numOfCameras):
#   pclFrames[i] = C.addFrame('pcl'+str(i+1), 'world')
#   pclFrames[i].setPointCloud(pcl[i], pclColor)
# C.view()

### Now we extract the ones inside the bounding box

In [ ]:
centre = C.getFrame("bin").getPosition()
centre[2] = centre[2] + 0.083  # Raise slightly to remove bin surface

off = .1
offsets = [0.3, 0.25, 0.05]

bounding_box_min_x = centre[0] - offsets[0]
bounding_box_max_x = centre[0] + offsets[0]

bounding_box_min_y = centre[1] - offsets[1]
bounding_box_max_y = centre[1] + offsets[1]

bounding_box_min_z = centre[2] - offsets[2]
bounding_box_max_z = centre[2] + offsets[2]


for i in range(numOfCameras):
  for h in range(pcl[i].shape[0]):  
    for w in range(pcl[i].shape[1]):  
      point = pcl[i][h, w] 
      eachX, eachY, eachZ = point  
      
      # Check if the point is within the bounding box
      if not (bounding_box_min_x < eachX < bounding_box_max_x and
              bounding_box_min_y < eachY < bounding_box_max_y and
              bounding_box_min_z < eachZ < bounding_box_max_z):
          pcl[i][h, w] = [0, 0, 0]


### Now we can view the bounded point cloud

In [248]:
# pclFrames = [None] * numOfCameras

for i in range(numOfCameras):
  pclFrames[i] = C.addFrame('pcl'+str(i+1), 'world')
  pclFrames[i].setPointCloud(pcl[i], pclColor)
  
C.view()


0

In [249]:
# finalpcl = pcl[0] + pcl[1] + pcl[2] + pcl[3]
# finalpclFrame = C.addFrame('finalpclFrame', 'world')
# finalpclFrame.setPointCloud(finalpcl, [ 255, 0, 0])
# C.view()

# print("pcl1: ", pcl[0].shape)
# print("pcl2: ", pcl[1].shape)

# extract the point cloud inside bounding box
# boundingBox = [0.5, 0.5, 0.5, 0.6, 0.6, 0.6]

# print positions of pcl points
# for point in pcl[0]:
  # print(point.getPosition())
